In [1]:
# Importing Libraries
from keras.layers import Dense,Conv1D,MaxPooling1D,Flatten,Dropout

Using TensorFlow backend.


In [0]:
import pandas as pd
import numpy as np

In [3]:
!unzip HumanActivityRecognization.zip

Archive:  HumanActivityRecognization.zip
   creating: te_Inertial Signals/
  inflating: te_Inertial Signals/body_acc_x_test.txt  
  inflating: te_Inertial Signals/body_acc_y_test.txt  
  inflating: te_Inertial Signals/body_acc_z_test.txt  
  inflating: te_Inertial Signals/body_gyro_x_test.txt  
  inflating: te_Inertial Signals/body_gyro_y_test.txt  
  inflating: te_Inertial Signals/body_gyro_z_test.txt  
  inflating: te_Inertial Signals/total_acc_x_test.txt  
  inflating: te_Inertial Signals/total_acc_y_test.txt  
  inflating: te_Inertial Signals/total_acc_z_test.txt  
   creating: tr_Inertial Signals/
  inflating: tr_Inertial Signals/body_acc_x_train.txt  
  inflating: tr_Inertial Signals/body_acc_y_train.txt  
  inflating: tr_Inertial Signals/body_acc_z_train.txt  
  inflating: tr_Inertial Signals/body_gyro_x_train.txt  
  inflating: tr_Inertial Signals/body_gyro_y_train.txt  
  inflating: tr_Inertial Signals/body_gyro_z_train.txt  
  inflating: tr_Inertial Signals/total_acc_x_train.

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_fwf(filename,encoding='utf-8', delim_whitespace=True, header=None)

# Utility function to load the load
def trainload_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = 'tr_Inertial Signals/'+signal+'_train.txt'
        print(filename)
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

def testload_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = 'te_Inertial Signals/'+signal+'_test.txt'
        print(filename)
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = 'y_'+subset+'.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train  = trainload_signals('train')
    X_test = testload_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [0]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [0]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [125]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

tr_Inertial Signals/body_acc_x_train.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


tr_Inertial Signals/body_acc_y_train.txt
tr_Inertial Signals/body_acc_z_train.txt
tr_Inertial Signals/body_gyro_x_train.txt
tr_Inertial Signals/body_gyro_y_train.txt
tr_Inertial Signals/body_gyro_z_train.txt
tr_Inertial Signals/total_acc_x_train.txt
tr_Inertial Signals/total_acc_y_train.txt
tr_Inertial Signals/total_acc_z_train.txt
te_Inertial Signals/body_acc_x_test.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


te_Inertial Signals/body_acc_y_test.txt
te_Inertial Signals/body_acc_z_test.txt
te_Inertial Signals/body_gyro_x_test.txt
te_Inertial Signals/body_gyro_y_test.txt
te_Inertial Signals/body_gyro_z_test.txt
te_Inertial Signals/total_acc_x_test.txt
te_Inertial Signals/total_acc_y_test.txt
te_Inertial Signals/total_acc_z_test.txt


In [17]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)

print(len(X_train))
print(n_classes)

128
9
7352
6


## 1. SIMPLE LSTM ONLY

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim),return_sequences=True))
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.4))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='softmax',kernel_initializer='glorot_normal'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 128, 32)           5376      
_________________________________________________________________
dropout_11 (Dropout)         (None, 128, 32)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 390       
Total params: 30,598
Trainable params: 30,598
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 73s 10ms/step - loss: 1.0560 - acc: 0.5529 - val_loss: 0.8554 - val_acc: 0.6379
Epoch 2/30
7352/7352 [==============================] - 69s 9ms/step - loss: 0.7018 - acc: 0.6921 - val_loss: 0.8546 - val_acc: 0.6295
Epoch 3/30
7352/7352 [==============================] - 69s 9ms/step - loss: 0.6713 - acc: 0.7018 - val_loss: 0.6700 - val_acc: 0.7628
Epoch 4/30
7352/7352 [==============================] - 69s 9ms/step - loss: 0.4615 - acc: 0.8456 - val_loss: 0.4022 - val_acc: 0.8527
Epoch 5/30
7352/7352 [==============================] - 69s 9ms/step - loss: 0.2721 - acc: 0.9081 - val_loss: 0.3979 - val_acc: 0.8728
Epoch 6/30
7352/7352 [==============================] - 69s 9ms/step - loss: 0.2492 - acc: 0.9142 - val_loss: 0.3905 - val_acc: 0.8700
Epoch 7/30
7352/7352 [==============================] - 70s 9ms/step - loss: 0.2413 - acc: 0.9193 - val_loss: 0.3230 - val_acc: 0.8904
Epoch 

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 619us/step


In [0]:
score

[0.46949227090775086, 0.8931116389548693]

##2. CONV1D + LSTM

In [0]:
# Initiliazing the sequential model
from keras.layers import Conv1D,BatchNormalization
from keras.regularizers import l2
model = Sequential()
model.add(Conv1D(input_shape=(timesteps,input_dim),
                 filters=64,
                 kernel_size=5,
                 padding='valid',
                 activation='relu',
                 strides=1,
                 kernel_initializer='glorot_normal', kernel_regularizer=l2(0.)))
# Configuring the parameters
model.add(Conv1D(filters=128,kernel_size=5,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.)))
model.add(BatchNormalization())
model.add(LSTM(n_hidden, return_sequences=True))
# Adding a dropout layer
model.add(LSTM(64))
model.add(Dropout(0.4))
# Adding a dense output layer with sigmoid activation
model.add(Dense(32,activation='relu',kernel_initializer='glorot_normal'))
model.add(Dense(n_classes, activation='softmax',kernel_initializer='glorot_normal'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 124, 64)           2944      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 120, 128)          41088     
_________________________________________________________________
batch_normalization_1 (Batch (None, 120, 128)          512       
_________________________________________________________________
lstm_2 (LSTM)                (None, 120, 32)           20608     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
__________

In [0]:
# Initiliazing the sequential model
from keras.layers import Flatten,Conv1D,BatchNormalization
from keras.regularizers import l2
model = Sequential()
model.add(Dense(500,activation='relu',kernel_initializer='glorot_uniform',input_shape=(128,9)))
model.add(Dense(250,activation='relu',kernel_initializer='glorot_uniform',kernel_regularizer=l2(0.5)))
model.add(Dense(100,activation='relu',kernel_initializer='glorot_uniform'))
model.add(Dense(32,activation='relu',kernel_initializer='glorot_uniform'))
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax',kernel_initializer='glorot_uniform'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_86 (Dense)             (None, 128, 500)          5000      
_________________________________________________________________
dense_87 (Dense)             (None, 128, 250)          125250    
_________________________________________________________________
dense_88 (Dense)             (None, 128, 100)          25100     
_________________________________________________________________
dense_89 (Dense)             (None, 128, 32)           3232      
_________________________________________________________________
flatten_3 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_90 (Dense)             (None, 6)                 24582     
Total params: 183,164
Trainable params: 183,164
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=10,
          validation_data=(X_test, Y_test),
          epochs=35)

Train on 7352 samples, validate on 2947 samples
Epoch 1/35
7352/7352 [==============================] - 5s 634us/step - loss: 6.5677 - acc: 0.7189 - val_loss: 0.7913 - val_acc: 0.7798
Epoch 2/35
7352/7352 [==============================] - 3s 475us/step - loss: 0.5195 - acc: 0.8439 - val_loss: 0.8704 - val_acc: 0.7954
Epoch 3/35
7352/7352 [==============================] - 3s 473us/step - loss: 0.4173 - acc: 0.8774 - val_loss: 0.7522 - val_acc: 0.8052
Epoch 4/35
7352/7352 [==============================] - 3s 473us/step - loss: 0.3761 - acc: 0.9015 - val_loss: 0.5054 - val_acc: 0.8660
Epoch 5/35
7352/7352 [==============================] - 4s 479us/step - loss: 0.3209 - acc: 0.9132 - val_loss: 0.6254 - val_acc: 0.8592
Epoch 6/35
7352/7352 [==============================] - 4s 482us/step - loss: 0.2957 - acc: 0.9154 - val_loss: 0.5277 - val_acc: 0.8690
Epoch 7/35
7352/7352 [==============================] - 4s 477us/step - loss: 0.2725 - acc: 0.9236 - val_loss: 0.6547 - val_acc: 0.8463


In [0]:
input_dim

## 3. CONV2D + LSTM  + DROPOUT

In [0]:
# Initiliazing the sequential model
from keras.layers import Conv2D,BatchNormalization,Conv1D,Flatten,Reshape
from keras.regularizers import l2
model = Sequential()
model.add(Conv2D(input_shape=(timesteps,input_dim,1),
                 filters=10,
                 kernel_size=3,
                 padding='valid',
                 activation='relu',
                 strides=(1,1),
                 kernel_initializer='glorot_normal'))
# Configuring the parameters
model.add(Conv2D(filters=128,kernel_size=5,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Reshape((122,3*128)))
model.add(LSTM(n_hidden, return_sequences=True))
# Adding a dropout layer
model.add(LSTM(64))
model.add(Dropout(0.4))
# Adding a dense output layer with sigmoid activation
model.add(Dense(32,activation='relu',kernel_initializer='glorot_normal'))
model.add(Dense(n_classes, activation='softmax',kernel_initializer='glorot_normal'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 126, 7, 10)        100       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 122, 3, 128)       32128     
_________________________________________________________________
batch_normalization_11 (Batc (None, 122, 3, 128)       512       
_________________________________________________________________
flatten_8 (Flatten)          (None, 46848)             0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 122, 384)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 122, 32)           53376     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                24832     
__________

In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

W0711 10:17:00.146879 140573411317632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
X_tr=X_train
X_te=X_test

In [0]:
X_tr = X_tr.reshape(7352,128,9,1)
X_te = X_te.reshape(2947,128,9,1)

In [0]:
# Training the model
model.fit(X_tr,
          Y_train,
          batch_size=1000,
          validation_data=(X_te, Y_test),
          epochs=15)

Train on 7352 samples, validate on 2947 samples
Epoch 1/15
7352/7352 [==============================] - 71s 10ms/step - loss: 0.5710 - acc: 0.7821 - val_loss: 0.6112 - val_acc: 0.7523
Epoch 2/15
7352/7352 [==============================] - 69s 9ms/step - loss: 0.5310 - acc: 0.7946 - val_loss: 0.6100 - val_acc: 0.7581
Epoch 3/15
7352/7352 [==============================] - 69s 9ms/step - loss: 0.4822 - acc: 0.8228 - val_loss: 0.5536 - val_acc: 0.7838
Epoch 4/15
7352/7352 [==============================] - 68s 9ms/step - loss: 0.4553 - acc: 0.8410 - val_loss: 0.5234 - val_acc: 0.7991
Epoch 5/15
7352/7352 [==============================] - 68s 9ms/step - loss: 0.4310 - acc: 0.8448 - val_loss: 0.5214 - val_acc: 0.7947
Epoch 6/15
7352/7352 [==============================] - 68s 9ms/step - loss: 0.3962 - acc: 0.8625 - val_loss: 0.4988 - val_acc: 0.8015
Epoch 7/15
7352/7352 [==============================] - 68s 9ms/step - loss: 0.3847 - acc: 0.8675 - val_loss: 0.5181 - val_acc: 0.7923
Epoch 

## 4. CONV2D + LSTM

In [0]:
# Initiliazing the sequential model
from keras.layers import Conv2D,BatchNormalization,Conv1D,Flatten,Reshape
from keras.regularizers import l2
model = Sequential()
model.add(Conv2D(input_shape=(timesteps,input_dim,1),
                 filters=128,
                 kernel_size=3,
                 padding='valid',
                 activation='relu',
                 strides=(1,1),
                 kernel_initializer='glorot_normal'))
# Configuring the parameters
#model.add(BatchNormalization())
model.add(Reshape((126,7*128)))
model.add(LSTM(n_hidden))
# Adding a dense output layer with sigmoid activation
model.add(Dense(32,activation='relu',kernel_initializer='glorot_normal'))
model.add(Dense(n_classes, activation='softmax',kernel_initializer='glorot_normal'))
model.summary()
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#Training the model

model.fit(X_tr,
          Y_train,
          batch_size=400,
          validation_data=(X_te, Y_test),
          epochs=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 126, 7, 128)       1280      
_________________________________________________________________
reshape_13 (Reshape)         (None, 126, 896)          0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 32)                118912    
_________________________________________________________________
dense_13 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_14 (Dense)             (None, 6)                 198       
Total params: 121,446
Trainable params: 121,446
Non-trainable params: 0
_________________________________________________________________
Train on 7352 samples, validate on 2947 samples
Epoch 1/50
7352/7352 [==============================] - 63s 9ms/step - loss: 1.6038 - ac

## 5.  LSTM + CONV2D

In [0]:
# Initiliazing the sequential model
from keras.layers import Conv2D,BatchNormalization,Conv1D,Flatten,Reshape
from keras.regularizers import l2
model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim),return_sequences=True))

# Configuring the parameters
#model.add(BatchNormalization())
model.add(Reshape((128,32,1)))
#model.add(LSTM(n_hidden))

model.add(Conv2D(filters=32,
                 kernel_size=3,
                 padding='valid',
                 activation='relu',
                 ))

# Adding a dense output layer with sigmoid activation
model.add(Flatten())
model.add(Dense(32,activation='relu',kernel_initializer='glorot_normal'))
model.add(Dense(n_classes, activation='softmax'))
model.summary()
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#Training the model

model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=30)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_27 (LSTM)               (None, 128, 32)           5376      
_________________________________________________________________
reshape_22 (Reshape)         (None, 128, 32, 1)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 126, 30, 32)       320       
_________________________________________________________________
flatten_17 (Flatten)         (None, 120960)            0         
_________________________________________________________________
dense_29 (Dense)             (None, 32)                3870752   
_________________________________________________________________
dense_30 (Dense)             (None, 6)                 198       
Total params: 3,876,646
Trainable params: 3,876,646
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Initiliazing the sequential model
from keras.layers import Conv2D,BatchNormalization,Conv1D,Flatten,Reshape
from keras.regularizers import l2
model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim),return_sequences=True))

# Configuring the parameters
#model.add(BatchNormalization())
model.add(Reshape((128,32,1)))
#model.add(LSTM(n_hidden))

model.add(Conv2D(filters=128,
                 kernel_size=3,
                 padding='valid',
                 activation='relu',
                 strides=(1,1),
                 kernel_initializer='glorot_normal'))

# Adding a dense output layer with sigmoid activation
model.add(Flatten())
model.add(Dense(32,activation='relu',kernel_initializer='glorot_normal'))
model.add(Dense(n_classes, activation='softmax',kernel_initializer='glorot_normal'))
model.summary()
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#Training the model

model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=30)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 128, 32)           5376      
_________________________________________________________________
reshape_17 (Reshape)         (None, 128, 32, 1)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 126, 30, 128)      1280      
_________________________________________________________________
flatten_14 (Flatten)         (None, 483840)            0         
_________________________________________________________________
dense_23 (Dense)             (None, 32)                15482912  
_________________________________________________________________
dense_24 (Dense)             (None, 6)                 198       
Total params: 15,489,766
Trainable params: 15,489,766
Non-trainable params: 0
________________________________________________________________

<h1>6. MLP

In [0]:
  # Initiliazing the sequential model
from keras.layers import Conv2D,BatchNormalization,Conv1D,Flatten,Reshape,MaxPooling2D
from keras.regularizers import l2
model = Sequential()
model.add(Conv2D(input_shape=(timesteps,input_dim,1),
                 filters=128,
                 kernel_size=2,
                 padding='valid',
                 activation='relu',
                 strides=(1,1),
                 kernel_initializer='glorot_normal'))
# Configuring the parameters
#model.add(BatchNormalization())
model.add(Conv2D(filters=60,kernel_size=2,activation='relu'))
model.add(Conv2D(filters=30,kernel_size=2,activation='relu'))
model.add(Conv2D(filters=20,kernel_size=2,activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
print(model.output.shape)
model.add(Reshape((62,40)))
model.add(LSTM(n_hidden,return_sequences=True))
model.add(LSTM(16))
# Adding a dense output layer with sigmoid activation
#model.add(Dense(32,activation='relu',kernel_initializer='glorot_normal'))
model.add(Dense(n_classes, activation='softmax',kernel_initializer='glorot_normal'))
model.summary()
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#Training the model
X_tr = X_train.reshape(7352,128,9,1)
X_te = X_test.reshape(2947,128,9,1)
model.fit(X_tr,
          Y_train,
          batch_size=20,
          validation_data=(X_te, Y_test),
          epochs=30)

(?, 62, 2, 20)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_66 (Conv2D)           (None, 127, 8, 128)       640       
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 126, 7, 60)        30780     
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 125, 6, 30)        7230      
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 124, 5, 20)        2420      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 62, 2, 20)         0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 62, 2, 20)         0         
_________________________________________________________________
reshape_14 (Reshape)         (None, 62, 40)            0     

<h1> 7.

In [0]:
  # Initiliazing the sequential model
from keras.layers import Conv2D,BatchNormalization,Conv1D,Flatten,Reshape,MaxPooling2D,CuDNNLSTM
from keras.models import Sequential
from keras.regularizers import l2
model = Sequential()
model.add(CuDNNLSTM(256, input_shape=(timesteps, input_dim), return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(CuDNNLSTM(256))
model.add(Dropout(0.1))
model.add(Dense(5, activation='sigmoid'))
model.add(Dense(n_classes, activation='softmax',kernel_initializer='glorot_normal'))
model.summary()
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#Training the model
X_tr = X_train.reshape(7352,128,9,1)
X_te = X_test.reshape(2947,128,9,1)
model.fit(X_train,
          Y_train,
          batch_size=20,
          validation_data=(X_test, Y_test),
          epochs=30)

W0713 07:23:57.086124 139838268270336 deprecation_wrapper.py:119] From /home/jupyter/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0713 07:24:03.506782 139838268270336 deprecation_wrapper.py:119] From /home/jupyter/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0713 07:24:04.072342 139838268270336 deprecation_wrapper.py:119] From /home/jupyter/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0713 07:24:04.082223 139838268270336 deprecation.py:506] From /home/jupyter/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 128, 256)          273408    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 256)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 256)          1024      
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 256)               526336    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1285      
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 36        
Total para

W0713 07:24:05.019942 139838268270336 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' used by node cu_dnnlstm_1/CudnnRNN (defined at /home/jupyter/.local/lib/python3.5/site-packages/keras/layers/cudnn_recurrent.py:517) with these attrs: [dropout=0, seed=42, input_mode="linear_input", T=DT_FLOAT, direction="unidirectional", rnn_mode="lstm", is_training=true, seed2=0]
Registered devices: [CPU, XLA_CPU]
Registered kernels:
  <no registered kernels>

	 [[cu_dnnlstm_1/CudnnRNN]]

Errors may have originated from an input operation.
Input Source operations connected to node cu_dnnlstm_1/CudnnRNN:
 cu_dnnlstm_1/ExpandDims_2 (defined at /home/jupyter/.local/lib/python3.5/site-packages/keras/layers/cudnn_recurrent.py:488)	
 cu_dnnlstm_1/ExpandDims_1 (defined at /home/jupyter/.local/lib/python3.5/site-packages/keras/layers/cudnn_recurrent.py:487)	
 cu_dnnlstm_1/transpose (defined at /home/jupyter/.local/lib/python3.5/site-packages/keras/layers/cudnn_recurrent.py:484)	
 cu_dnnlstm_1/concat_1 (defined at /home

In [0]:
from keras.models import Sequential

##HyperParameter Tuning

In [0]:
# Initiliazing the sequential model
def create_model(neurons=1):
  model = Sequential()
  # Configuring the parameters
  model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim),return_sequences=True))
  # Adding a dropout layer
  model.add(LSTM(128))
  model.add(Dropout(0.5))
  # Adding a dense output layer with sigmoid activation
  model.add(Dense(neurons,activation='relu',kernel_initializer='glorot_normal'))
  model.add(Dense(n_classes, activation='softmax',kernel_initializer='glorot_normal'))
  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [0]:
#https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=create_model, verbose=5)
batch_size = [10, 20, 40]
epochs = [10, 15, 20]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid,verbose=5, n_jobs=27)
grid_result = grid.fit(X_train, Y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=27)]: Using backend LokyBackend with 27 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=27)]: Done   4 out of  27 | elapsed: 169.4min remaining: 974.0min
[Parallel(n_jobs=27)]: Done  10 out of  27 | elapsed: 210.8min remaining: 358.4min
[Parallel(n_jobs=27)]: Done  16 out of  27 | elapsed: 248.4min remaining: 170.8min
[Parallel(n_jobs=27)]: Done  22 out of  27 | elapsed: 293.3min remaining: 66.7min
[Parallel(n_jobs=27)]: Done  27 out of  27 | elapsed: 315.2min fini

Epoch 1/15
Epoch 2/15
Epoch 3/15
Epoch 4/15
Epoch 5/15
Epoch 6/15
Epoch 7/15
Epoch 8/15
Epoch 9/15
Epoch 10/15
Epoch 11/15
Epoch 12/15
Epoch 13/15
Epoch 14/15
Epoch 15/15
Best: 0.854325 using {'batch_size': 10, 'epochs': 15}
0.736398 (0.078237) with: {'batch_size': 10, 'epochs': 10}
0.854325 (0.064620) with: {'batch_size': 10, 'epochs': 15}
0.729461 (0.146629) with: {'batch_size': 10, 'epochs': 20}
0.609630 (0.101199) with: {'batch_size': 20, 'epochs': 10}
0.687976 (0.124422) with: {'batch_size': 20, 'epochs': 15}
0.773939 (0.181414) with: {'batch_size': 20, 'epochs': 20}
0.769178 (0.090829) with: {'batch_size': 40, 'epochs': 10}
0.805767 (0.077979) with: {'batch_size': 40, 'epochs': 15}
0.794070 (0.047790) with: {'batch_size': 40, 'epochs': 20}


In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

In [0]:
#https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=create_model,batch_size=10,epochs=15, verbose=5)
init_mode = ['glorot_normal', 'glorot_uniform', 'he_normal']
param_grid = dict(init_mode = init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid,verbose=5, n_jobs=9)
grid_result = grid.fit(X_train, Y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=9)]: Using backend LokyBackend with 9 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=9)]: Done   2 out of   9 | elapsed: 163.6min remaining: 572.6min
[Parallel(n_jobs=9)]: Done   4 out of   9 | elapsed: 164.2min remaining: 205.2min
[Parallel(n_jobs=9)]: Done   6 out of   9 | elapsed: 167.4min remaining: 83.7min
[Parallel(n_jobs=9)]: Done   9 out of   9 | elapsed: 168.2min finished
W0706 22:17:40.830558 140084044724096 deprecation_wrapper.py:119] From /usr/local/li

Epoch 1/15
Epoch 2/15
Epoch 3/15
Epoch 4/15
Epoch 5/15
Epoch 6/15
Epoch 7/15
Epoch 8/15
Epoch 9/15
Epoch 10/15
Epoch 11/15
Epoch 12/15
Epoch 13/15
Epoch 14/15
Epoch 15/15
Best: 0.870783 using {'init_mode': 'glorot_normal'}
0.870783 (0.036987) with: {'init_mode': 'glorot_normal'}
0.782372 (0.061120) with: {'init_mode': 'glorot_uniform'}
0.860854 (0.083082) with: {'init_mode': 'he_normal'}


In [0]:
#https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=create_model,batch_size=10,epochs=15, verbose=5)
neurons = [32,64,128]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid,verbose=5, n_jobs=9)
grid_result = grid.fit(X_train, Y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=9)]: Using backend LokyBackend with 9 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Using Hyperas

In [0]:
n_hidden=128

In [0]:
# Initiliazing the sequential model
from keras.layers import Conv2D,BatchNormalization,Conv1D,Flatten,Reshape
from keras.regularizers import l2
def cmodel(X_train, Y_train, X_test, Y_test):
  model = Sequential()
  model.add(LSTM(32, input_shape=(128, 9),return_sequences=True))

  # Configuring the parameters
  #model.add(BatchNormalization())
  model.add(Reshape((128,32,1)))
  #model.add(LSTM(n_hidden))

  model.add(Conv2D(filters=32,
                   kernel_size={{choice([3,5])}},
                   padding='valid',
                   activation='relu',
                   ))

  # Adding a dense output layer with sigmoid activation
  model.add(Flatten())
  model.add(Dense({{choice([512, 1024])}},activation='relu',kernel_initializer='glorot_normal'))
  model.add(Dense(6, activation='softmax'))
  # Compiling the model
  #print("X_train---------------------------------------",X_train)
  #adam = Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
  
  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

  #Training the model
  model.fit(X_train,
            Y_train,
            batch_size={{choice([15,30])}},
            validation_data=(X_test, Y_test),
            epochs=8)
  score, acc = model.evaluate(X_test, Y_test, verbose=0)
  
    #print('Test accuracy:', acc)
  return { 'loss':-acc,'status': STATUS_OK, 'model': model}

In [0]:
X_train, X_test,Y_train, Y_test = load_data()

def data():
  X_train=np.load('a.npy')
  X_test=np.load('b.npy')
  Y_train=np.load('c.npy')
  Y_test=np.load('d.npy')
  return X_train,Y_train,X_test,Y_test

best_run, best_model = optim.minimize(model=cmodel,
                                      data=data,
                                      algo=tpe.suggest,                               
                                      max_evals=3,verbose=False,
                                      trials=Trials(),notebook_name='CopyHAR_LSTM')
                                      

train_Signals/body_acc_x_train.txt


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


train_Signals/body_acc_y_train.txt
train_Signals/body_acc_z_train.txt
train_Signals/body_gyro_x_train.txt
train_Signals/body_gyro_y_train.txt
train_Signals/body_gyro_z_train.txt
train_Signals/total_acc_x_train.txt
train_Signals/total_acc_y_train.txt
train_Signals/total_acc_z_train.txt
test_Signals/body_acc_x_test.txt


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


test_Signals/body_acc_y_test.txt
test_Signals/body_acc_z_test.txt
test_Signals/body_gyro_x_test.txt
test_Signals/body_gyro_y_test.txt
test_Signals/body_gyro_z_test.txt
test_Signals/total_acc_x_test.txt
test_Signals/total_acc_y_test.txt
test_Signals/total_acc_z_test.txt


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


Train on 7352 samples, validate on 2947 samples
Epoch 1/8
  15/7352 [..............................]
 - ETA: 1:04:19 - loss: 1.7816 - acc: 0.3333

  30/7352 [..............................]
 - ETA: 36:23 - loss: 2.4475 - acc: 0.3667  

  45/7352 [..............................]
 - ETA: 27:20 - loss: 2.8543 - acc: 0.3333

  60/7352 [..............................]
 - ETA: 22:50 - loss: 2.4711 - acc: 0.3833

  75/7352 [..............................]
 - ETA: 20:07 - loss: 2.1947 - acc: 0.4267

  90/7352 [..............................]
 - ETA: 18:21 - loss: 2.0054 - acc: 0.4333

 105/7352 [..............................]
 - ETA: 17:04 - loss: 2.0044 - acc: 0.4286

 120/7352 [..............................]
 - ETA: 16:04 - loss: 1.8629 - acc: 0.4833

 135/7352 [..............................]
 - ETA: 15:13 - loss: 1.8538 - acc: 0.4741

 150/7352 [..............................]
 - ETA: 14:36 - loss: 1.8065 - acc: 0.5000

 165/7352 [..............................]
 - ETA: 14:05 - loss: 1.7

In [0]:
best_run

## 7. DIVIDE AND CONQUER

In [0]:
Y_train_class=np.argmax(Y_train,axis=1) + 1
Y_test_class=np.argmax(Y_test,axis=1) + 1

In [0]:
Y_train_class[Y_train_class<=3] = 0
Y_train_class[Y_train_class>3] = 1

In [0]:
Y_test_class[Y_test_class<=3] = 0
Y_test_class[Y_test_class>3]  = 1

In [134]:
Y_train_class

array([1, 1, 1, ..., 0, 0, 0])

In [135]:
Y_train_class=pd.get_dummies(Y_train_class).as_matrix()
Y_test_class=pd.get_dummies(Y_test_class).as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [97]:
Y_train_class.shape,Y_test_class.shape

((7352, 4), (2947, 6))

In [0]:
from sklearn.preprocessing import StandardScaler
ttsd=StandardScaler()

ttsd.fit(X_train.reshape(X_train.shape[0] * X_train.shape[1], X_train.shape[2]))
x_train = ttsd.transform(X_train.reshape(X_train.shape[0] * X_train.shape[1], X_train.shape[2]))
x_test = ttsd.fit_transform(X_test.reshape(X_test.shape[0] * X_test.shape[1], X_test.shape[2]))



In [137]:
X_train.shape

(7352, 128, 9)

In [0]:
x_train = x_train.reshape(X_train.shape)
x_test = x_test.reshape(X_test.shape)

In [140]:
# Initiliazing the sequential model
model1 = Sequential()
model1.add(Conv1D(input_shape=(timesteps,input_dim),filters=64, kernel_size=3, activation='relu',kernel_initializer='glorot_normal'))
model1.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='glorot_normal'))
model1.add(Dropout(0.51))
model1.add(MaxPooling1D(pool_size=2))
model1.add(Flatten())
model1.add(Dense(50, activation='relu'))
model1.add(Dense(2, activation='softmax'))
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_17 (Conv1D)           (None, 126, 64)           1792      
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 124, 32)           6176      
_________________________________________________________________
dropout_9 (Dropout)          (None, 124, 32)           0         
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 62, 32)            0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 1984)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 50)                99250     
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 102       
Total para

In [0]:
# Compiling the model
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [143]:
# Training the model
model1.fit(x_train,
          Y_train_class,
          batch_size=batch_size,
          validation_data=(x_test, Y_test_class),
          epochs=10)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 3s 383us/step - loss: 0.0222 - acc: 0.9921 - val_loss: 0.0106 - val_acc: 0.9969
Epoch 2/10
7352/7352 [==============================] - 2s 294us/step - loss: 0.0015 - acc: 0.9995 - val_loss: 0.0087 - val_acc: 0.9986
Epoch 3/10
7352/7352 [==============================] - 2s 301us/step - loss: 5.2746e-06 - acc: 1.0000 - val_loss: 0.0336 - val_acc: 0.9952
Epoch 4/10
7352/7352 [==============================] - 2s 301us/step - loss: 2.4878e-04 - acc: 0.9999 - val_loss: 0.0122 - val_acc: 0.9986
Epoch 5/10
7352/7352 [==============================] - 2s 298us/step - loss: 4.2237e-05 - acc: 1.0000 - val_loss: 0.0154 - val_acc: 0.9980
Epoch 6/10
7352/7352 [==============================] - 2s 299us/step - loss: 1.1940e-07 - acc: 1.0000 - val_loss: 0.0167 - val_acc: 0.9980
Epoch 7/10
7352/7352 [==============================] - 2s 300us/step - loss: 1.1923e-07 - acc: 1.0000 - val_loss: 0.020

In [0]:
model1.save('dynamic_static_model.h5')

In [50]:
score = model.evaluate(X_test, Y_test_class)

2947/2947 [==============================] - 0s 74us/step


In [0]:
score
Y_test_class

In [0]:
z[:500]

- With a simple layer architecture to distinguish between static and dynamic activities we got 99% accuracy and a loss of 0.01


In [145]:

z=model1.predict(x_test)
z=np.argmax(z,axis=1)
z

array([1, 1, 1, ..., 0, 0, 0])

<h3>Static Activities 

In [0]:
Y_train_class=np.argmax(Y_train,axis=1)+1
Y_test_class=np.argmax(Y_test,axis=1)+1

In [0]:
X_train_static = X_train[Y_train_class > 3]
X_test_static = X_test[Y_test_class > 3]

In [0]:
Y_train_static = Y_train_class[Y_train_class>3]
Y_test_static = Y_test_class[Y_test_class>3]

In [41]:
Y_train_static = pd.get_dummies(Y_train_static).as_matrix()
Y_test_static  = pd.get_dummies(Y_test_static).as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [42]:
X_train_static.shape,X_test_static.shape

((4067, 128, 9), (1560, 128, 9))

In [0]:
from sklearn.preprocessing import StandardScaler
tts=StandardScaler()

tts.fit(X_train_static.reshape((X_train_static.shape[0] * X_train_static.shape[1], X_train_static.shape[2])))
x_train = tts.transform(X_train_static.reshape((X_train_static.shape[0] * X_train_static.shape[1], X_train_static.shape[2])))
x_test = tts.fit_transform(X_test_static.reshape((X_test_static.shape[0] * X_test_static.shape[1], X_test_static.shape[2])))

In [0]:
X_train_static = x_train.reshape(X_train_static.shape)
X_test_static = x_test.reshape(X_test_static.shape)

In [0]:
from keras.regularizers import l2
model2 = Sequential()
model2.add(Conv1D(filters=32,kernel_size=5,activation='relu',
                    kernel_initializer='he_normal',
                    kernel_regularizer=l2(0.001),input_shape=(128,9)))
model2.add(Conv1D(filters=16, kernel_size=3, 
                activation='relu',kernel_regularizer=l2(0.001),kernel_initializer='he_normal'))

model2.add(Dropout(0.4))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Flatten())
model2.add(Dense(64, activation='relu'))
model2.add(Dense(3, activation='softmax'))

In [47]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 124, 32)           1472      
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 122, 16)           1552      
_________________________________________________________________
dropout_4 (Dropout)          (None, 122, 16)           0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 61, 16)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 976)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                62528     
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 195       
Total para

In [48]:
Y_test_static.shape

(1560, 3)

In [49]:
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
result = model2.fit(X_train_static, Y_train_static,
                    batch_size=64,
                    epochs=39,
                    validation_data=(X_test_static, Y_test_static))

Train on 4067 samples, validate on 1560 samples
Epoch 1/39
4067/4067 [==============================] - 1s 269us/step - loss: 0.5271 - acc: 0.8286 - val_loss: 0.4441 - val_acc: 0.8590
Epoch 2/39
4067/4067 [==============================] - 0s 89us/step - loss: 0.3045 - acc: 0.9083 - val_loss: 0.4541 - val_acc: 0.8712
Epoch 3/39
4067/4067 [==============================] - 0s 90us/step - loss: 0.2707 - acc: 0.9218 - val_loss: 0.3905 - val_acc: 0.8737
Epoch 4/39
4067/4067 [==============================] - 0s 89us/step - loss: 0.2494 - acc: 0.9307 - val_loss: 0.3043 - val_acc: 0.9205
Epoch 5/39
4067/4067 [==============================] - 0s 89us/step - loss: 0.2318 - acc: 0.9410 - val_loss: 0.3173 - val_acc: 0.9199
Epoch 6/39
4067/4067 [==============================] - 0s 85us/step - loss: 0.2096 - acc: 0.9444 - val_loss: 0.3259 - val_acc: 0.9250
Epoch 7/39
4067/4067 [==============================] - 0s 98us/step - loss: 0.1993 - acc: 0.9506 - val_loss: 0.2868 - val_acc: 0.9340
Epoch 

**Validation accuracy is 93% for for classification of static activities**

In [0]:
model2.save('model_static.h5')

In [0]:
z=model2.predict(X_test_static)
z=np.argmax(z,axis=1)
z[:500]

<h3>Dynamic Activities

In [0]:
Y_train_class=np.argmax(Y_train,axis=1)+1
Y_test_class=np.argmax(Y_test,axis=1)+1

X_train_dynamic = X_train[Y_train_class <= 3]
X_test_dynamic = X_test[Y_test_class <= 3]

In [56]:
Y_train_dynamic = Y_train_class[Y_train_class <= 3]
Y_test_dynamic = Y_test_class[Y_test_class <= 3]

Y_train_dynamic = pd.get_dummies(Y_train_dynamic).as_matrix()
Y_test_dynamic  = pd.get_dummies(Y_test_dynamic).as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [0]:
from sklearn.preprocessing import StandardScaler
ttd=StandardScaler()

x_train=ttd.fit(X_train_dynamic.reshape((X_train_dynamic.shape[0] * X_train_dynamic.shape[1], X_train_dynamic.shape[2])))
x_train=ttd.fit_transform(X_train_dynamic.reshape((X_train_dynamic.shape[0] * X_train_dynamic.shape[1], X_train_dynamic.shape[2])))
x_test=ttd.fit_transform(X_test_dynamic.reshape((X_test_dynamic.shape[0] * X_test_dynamic.shape[1], X_test_dynamic.shape[2])))

X_train_dynamic = x_train.reshape(X_train_dynamic.shape)
X_test_dynamic = x_test.reshape(X_test_dynamic.shape)

**Model**

In [0]:
from keras.regularizers import l2
model3 = Sequential()
model3.add(Conv1D(filters=32,kernel_size=5,activation='relu',
                    kernel_initializer='he_normal',
                    kernel_regularizer=l2(0.001),input_shape=(128,9)))
model3.add(Conv1D(filters=16, kernel_size=3, 
                activation='relu',kernel_regularizer=l2(0.001),kernel_initializer='he_normal'))

model3.add(Dropout(0.4))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Flatten())
model3.add(Dense(64, activation='relu'))
model3.add(Dense(3, activation='softmax'))

In [60]:
model3.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
result = model3.fit(X_train_dynamic, Y_train_dynamic,
                    batch_size=64,
                    epochs=40,
                    validation_data=(X_test_dynamic, Y_test_dynamic))

Train on 3285 samples, validate on 1387 samples
Epoch 1/40
3285/3285 [==============================] - 1s 292us/step - loss: 1.1015 - acc: 0.6049 - val_loss: 0.5685 - val_acc: 0.8745
Epoch 2/40
3285/3285 [==============================] - 0s 103us/step - loss: 0.3100 - acc: 0.9269 - val_loss: 0.3785 - val_acc: 0.9034
Epoch 3/40
3285/3285 [==============================] - 0s 88us/step - loss: 0.1636 - acc: 0.9781 - val_loss: 0.3197 - val_acc: 0.9423
Epoch 4/40
3285/3285 [==============================] - 0s 86us/step - loss: 0.1357 - acc: 0.9890 - val_loss: 0.2300 - val_acc: 0.9654
Epoch 5/40
3285/3285 [==============================] - 0s 85us/step - loss: 0.1129 - acc: 0.9939 - val_loss: 0.2435 - val_acc: 0.9697
Epoch 6/40
3285/3285 [==============================] - 0s 95us/step - loss: 0.1004 - acc: 0.9982 - val_loss: 0.2626 - val_acc: 0.9647
Epoch 7/40
3285/3285 [==============================] - 0s 93us/step - loss: 0.0985 - acc: 0.9963 - val_loss: 0.3244 - val_acc: 0.9567
Epoch

**Validation accuracy is 97% for for classification of static activities**

In [0]:
model3.save('model_dynamic.h5')

In [0]:
z=model3.predict(X_test_dynamic)
z=np.argmax(z,axis=1)
z[:500]

In [0]:
from keras.models import load_model
model1 = load_model('dynamic_static_model.h5')
model3 = load_model('model_dynamic.h5')
model2 = load_model('model_static.h5')

**Validation Data**

In [174]:
predict_ds = model1.predict(x_test)
predict_ds = np.argmax(predict_ds, axis=1)

X_static = X_test[predict_ds == 1]
X_dynamic = X_test[predict_ds == 0]

x_static = X_static.reshape(X_static.shape[0]*X_static.shape[1],X_static.shape[2])

predict_static = model2.predict((tts.transform(x_static)).reshape(X_static.shape))
predict_static = np.argmax(predict_static,axis=1)+4
predict_static

array([5, 5, 5, ..., 6, 6, 6])

In [0]:
x_dynamic = X_dynamic.reshape(X_dynamic.shape[0]*X_dynamic.shape[1],X_dynamic.shape[2])

predict_dynamic = model3.predict((ttd.transform(x_dynamic)).reshape(X_dynamic.shape))
predict_dynamic = np.argmax(predict_dynamic,axis=1)+1

In [0]:
predict_ds

In [0]:
i,j = 0,0 
val_pred = []
for flag in predict_ds:
        if flag == 1:
            val_pred.append(predict_static[i])
            i = i + 1
        else:
            val_pred.append(predict_dynamic[j])
            j = j + 1 

In [0]:
Y_test_cls

In [0]:
Y_train_cls = np.argmax(Y_train,axis=1) + 1
Y_test_cls = np.argmax(Y_test,axis=1) + 1

In [177]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test_cls,val_pred)

0.9518154054971157

**Train Data**

In [0]:
predict_ds = model1.predict(x_train)
predict_ds = np.argmax(predict_ds, axis=1)

X_static = X_train[predict_ds == 1]
X_dynamic = X_train[predict_ds == 0]

x_static = X_static.reshape(X_static.shape[0]*X_static.shape[1],X_static.shape[2])

predict_static = model2.predict((tts.transform(x_static)).reshape(X_static.shape))
predict_static = np.argmax(predict_static,axis=1)+4


x_dynamic = X_dynamic.reshape(X_dynamic.shape[0]*X_dynamic.shape[1],X_dynamic.shape[2])

predict_dynamic = model3.predict((ttd.transform(x_dynamic)).reshape(X_dynamic.shape))
predict_dynamic = np.argmax(predict_dynamic,axis=1)+1

In [180]:
i,j = 0,0 
train_pred = []
for flag in predict_ds:
        if flag == 1:
            train_pred.append(predict_static[i])
            i = i + 1
        else:
            train_pred.append(predict_dynamic[j])
            j = j + 1 


from sklearn.metrics import accuracy_score
accuracy_score(Y_train_cls,train_pred)

0.9866702937976061

**Confusion Matrix**

In [185]:
from sklearn.metrics import confusion_matrix
c = confusion_matrix(Y_test_cls,val_pred)
print(c)

[[486   3   7   0   0   0]
 [  3 446  22   0   0   0]
 [  4   0 416   0   0   0]
 [  0   3   0 447  36   5]
 [  0   0   0  36 492   4]
 [  0  18   0   1   0 518]]


In [186]:
from sklearn.metrics import confusion_matrix
c = confusion_matrix(Y_train_cls,train_pred)
print(c)

[[1226    0    0    0    0    0]
 [   0 1073    0    0    0    0]
 [   0    0  986    0    0    0]
 [   0    0    0 1282    1    3]
 [   0    0    0   93 1281    0]
 [   0    0    0    1    0 1406]]


**Conclusion**<br>
First we tried random model to evaluate the prediction . Then we do the hyperparameter tuning for kernel initializer ,optimizer, no of epoch and the batch size . After finding this we have to try the different models with different droput rate to increase the performance.<br>
Then we used divide and conquer Method which worked really well

In [182]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["Model", "Train Accuracy", "Test Accuracy"]

x.add_row(["1. SIMPLE LSTM ONLY", 0.9450 , 0.9131])
x.add_row(["2. CONV1D + LSTM", 0.9528, 0.8870])
x.add_row(["3. CONV2D + LSTM + DROPOUT", 0.9191 , 0.8113])
x.add_row(["4. CONV2D + LSTM ", 0.9582 , 0.8863])
x.add_row(["5. LSTM + CONV2D", 0.9759 , 0.9175])
x.add_row(["6. DIVIDE AND CONQUER", 0.986 , 0.951])

print(x)

+----------------------------+----------------+---------------+
|           Model            | Train Accuracy | Test Accuracy |
+----------------------------+----------------+---------------+
|    1. SIMPLE LSTM ONLY     |     0.945      |     0.9131    |
|      2. CONV1D + LSTM      |     0.9528     |     0.887     |
| 3. CONV2D + LSTM + DROPOUT |     0.9191     |     0.8113    |
|     4. CONV2D + LSTM       |     0.9582     |     0.8863    |
|      5. LSTM + CONV2D      |     0.9759     |     0.9175    |
|   6. DIVIDE AND CONQUER    |     0.986      |     0.951     |
+----------------------------+----------------+---------------+


References : <br>
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5949027/<br>
https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/<br>
https://github.com/UdiBhaskar/Human-Activity-Recognition--Using-Deep-NN